In [15]:
# Make a gif of the plots in thedirectory
# /projectnb/eb-general/wade/sfno/inference/viz/figures/Experiment1/
import os
import imageio
import matplotlib.pyplot as plt

# Directory containing the images
image_dir = "/projectnb/eb-general/wade/sfno/inference/viz/figures/Experiment1/"
# Get a list of all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
# Sort the image files 
image_files.sort()
# Create a list to hold the images
images = []
# Read each image and append it to the list
for filename in image_files:
    image_path = os.path.join(image_dir, filename)
    images.append(imageio.imread(image_path))

# Save the images as a GIF
gif_path = os.path.join(image_dir, "experiment1_.gif")
imageio.mimsave(gif_path, images, duration=100)  # duration is the time between frames in seconds
print(f"GIF saved at: {gif_path}")


GIF saved at: /projectnb/eb-general/wade/sfno/inference/viz/figures/Experiment1/experiment1_.gif


/scratch/401934.1.ood/ipykernel_770926/785353439.py:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(image_path))
